In [ ]:
# ===============================================================
# 03_kerr_flux_sim.ipynb  —  Kerr-enhanced quaternionic flux (stable shooting)
# ===============================================================
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ---------- 常數與黑洞參數 ---------------------------------------
G, c = 6.67430e-11, 2.99792458e8
M_sun, M_pl = 1.98847e30, 2.176434e-8

M_bh   = 4.0e6 * M_sun
a_star = 0.5

r_g_m  = 2*G*M_bh/c**2
print(f"r_g ≈ {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ---------- 無因次化變數 ------------------------------------------
a_hat  = a_star
Mratio = M_bh / M_pl               # ≈ 3.65×10^44

Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

κ = 2.0e-2
λ_tilde = lambda x: 2*κ*Mratio / x**3   # curvature–driven coupling

def rhs(x, y):
    ψ, χ = y
    src  = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)
    # 一階項來自 (x^2 Δ ψ')' 展開
    pref = x**2 * Δ(x)
    lin  = (2*x*Δ(x) + x**2 * dΔ(x)) * χ
    χp   = (src - lin) / pref
    return [χ, χp]

# ---------- 外射擊法：從大 r̂ 向內 integrate(A) -------------------
x_outer = 1e3  # r̂=1000

def integrate(A):
    # 遠端近似 ψ'≈ -A/x^2, ψ≈ A/x
    ψ0, χ0 = A/x_outer, -A/x_outer**2
    sol = solve_ivp(
        rhs,
        t_span=(x_outer, 2.0),              # 從外往內 (停在 r̂=2)
        y0=[ψ0, χ0],
        t_eval=np.geomspace(x_outer, 2.0, 1500),
        method="Radau",
        rtol=1e-7, atol=1e-10,
        max_step=5.0
    )
    if not sol.success or sol.y.shape[1]==0:
        return None, None
    return sol.t[::-1], (2*sol.y[0])[::-1]   # 反轉成 r̂ 遞增順序

# ---------- 掃描 A 以命中 ε(10) ≈ target ------------------------
target = 0.1
As = np.logspace(-4, 1, 60)
best = None
for A in As:
    x, eps = integrate(A)
    if x is None:
        continue
    # 找最接近 r̂=10 的索引
    idx = np.argmin(np.abs(x - 10))
    err = abs(eps[idx] - target)
    if best is None or err < best[1]:
        best = (A, err)

if best is None:
    raise RuntimeError("所有 A 均無法積分成功，請調整範圍或步長！")
bestA, bestErr = best
print(f"Selected A = {bestA:.3e}  (ε(10) 偏差 {bestErr:.2e})")

# 用最佳 A 重算最終解
x_arr, eps_arr = integrate(bestA)

# ---------- 論文中的 analytic ansatz ----------------------------
r_s0_hat = (8.5e3 * 3.085677e16) / r_g_m  # 8.5 kpc → 無因次
boost    = 1.0 / (1 - a_hat/x_arr)
eps_ans  = 2*np.tanh(x_arr / (r_s0_hat * boost))

# ---------- 繪圖比較 ----------------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr, eps_arr,   lw=2, label="Numerical ε(r)")
plt.loglog(x_arr, eps_ans, '--', lw=2, label="Analytic ansatz")
plt.axvline(10, ls=':', color='k'); plt.text(10.2, 1e-1, "10 r_g")
plt.xlabel(r"$\hat r = r/r_g$"); plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime (shooting)")
plt.legend(); plt.grid(ls='--', alpha=.3); plt.tight_layout()
plt.show()

# ---------- 估算 δc_g/c at r̂=10 -------------------------------------
f_LISA = 1e-2  # Hz
k_hat  = 2*pi*f_LISA * (r_g_m/c)   # k 轉無因次
idx10  = np.argmin(np.abs(x_arr - 10))
epsilon10 = eps_arr[idx10]
delta_c   = (epsilon10**2 * (a_hat/10)**2) / (2*k_hat**2)
print(f"\nε(10 r_g) ≈ {epsilon10:.2e}")
print(f"δc_g/c     ≈ {delta_c:.2e}")
